In [486]:
#Imports
import numpy as np
import os
import base64
import hashlib
import bcrypt

In [494]:
#Admin account comes pre-installed on the database. This is how clearances and accounts are initially edited.
#Admin password is hashed, then the cleartext password is removed from memory 
#and the hashed password, along with the salt used to hash it, are stored in the database.
#Admin is the only account with a clearance of 5. They are the only person who can create accounts with clearances of 4.
adminSalt = bcrypt.gensalt()
adminPassword = "adminPassword"
adminHash = bcrypt.hashpw(adminPassword.encode(), adminSalt)
adminPassword = ""

database = {
    "admin": [adminHash,adminSalt,5]
}


#This implementation is code for the database computer not the user's computer.
#Encrypting passwords before sending them over the network using a Diffie-Helman key exchange and symmetric key 
#encryption is a necessary part of the security of the system.
#However, these methods assume that the password has already been encrypted, sent over, and decrypted.

#Method to add a user
def addUser(username, password, clearance):
     
    if(type(clearance) == int):#User input validation. Ensures that clearance is an int.
        if(username in database.keys()):#Checks to make sure the requested username does not already exist
            print("Username already taken by another user. Please choose a different username.")
        elif(clearance > 4 or clearance < 1): #Ensures that the requested clearance is a valid clearance level.
            print("The requested clearance must be greater than 0 and less than 5")
        else:
            for entry in database:
                if(database[entry][2] > clearance):
                    print(entry + ", would you like to set " + username + "'s clearance level? They requested a clearance of " + str(clearance) + ".")
                    #Sends a message to all users whose clearance is greater than the requested clearance. 
                    #Print is analogous to sending a message.
            salt = bcrypt.gensalt()
            hashEntry = bcrypt.hashpw(password.encode(),salt)
            database[username] = [hashEntry,salt,0]
            #Generates a salt, adds it to the password, and stores both the salt and the hashed password in the database.
            return True #Returns True if successful.
    else:
        print("Please request an integer clearance greater than 0 and less than 5.")
    return False #Returns False if unsuccessful.

#Method to authenticate a user. Returns their clearance level if the username and password are correct.
#Returns -1 if the username and password are not correct.
#Used in many of the other functions
def login(username, password):
    #Checks to make sure that your username actually exists.
    #Also authenticates the user by getting taking the password the entered, adding the salt, hashing that and 
    #comparing it to the hash stored in the databse.
    if(username in database.keys() and database[username][0] == bcrypt.hashpw(password.encode(),database[username][1])):
        return database[username][2] # Returns clearance level if the username and password are correct.
    else:
        print("Incorrect username or password.")
        return -1 # Returns -1 if the username and password are not correct.

#Method to change the clearance level of a user.
def changeClearanceLevel(myUsername, myPassword, theirUsername, theirNewClearance):
    if(theirUsername in database.keys()):#Checks to make sure that the user whose clearance you are trying to change actually exists
        myClearance = login(myUsername, myPassword)
        if(myClearance > -1):#Account is invalid if clearance returned is -1. Error message comes from login method.
            #User input validation: Checks to make sure that the new clearance you wish to assign to them is an integer
            #Also does not allow you to assign a clearance greater than or equal to your own.
            if(theirNewClearance >= 0 and theirNewClearance < myClearance and type(theirNewClearance) == int):
                database[theirUsername][2] = theirNewClearance
                return True #Returns True if successful.
            else: 
                print("Please select an integer clearance for the other user that is greater than -1 and less than " + str(myClearance) + ".")
    else:
        print("The user whose clearance you wish to change does not exist.")
    return False #Returns False if unsuccessful.

#Method to delete an account. Only usable by the admin.
def deleteUser(adminUsername, adminPassword, employeeUsername):
    #Authenticates the admin using login and checks the clearance level to veryify it is the admin. 
    #Also makes sure that the clearance level of the employee is not 5.
    #This means that the admin cannot accidentally delete their own account.
    if(login(adminUsername,adminPassword) == 5 and database[employeeUsername][2] < 5):
        del database[employeeUsername]
        return True #Returns True if successful.
    else:
        print("You do not have permission to delete this account.")
        return False #Returns False if unsuccessful.

#Function that lets users use addition, subtraction, and multiplication on integers depending upon their clearance level.
def intArithmetic(username, password, operation, n1, n2):
    #User input validation: Checks to make sure that the two numbers entered are integers
    if(type(n1) == int and type(n1) == int):
        allOperations = ["+","-","*"]
        #User input validation: Checks to make sure that the entered operation is a valid arithmetic operation.
        if(operation in allOperations):
            
            #Checks to make sure the username exists
            myClearance = login(username,password)
            if(myClearance > -1):#Account is invalid if clearance returned is -1. Error message comes from login method.
                requiredClearance = allOperations.index(operation) + 1#Gets the required clearance for the selected operation.
                if(myClearance >= requiredClearance): #Checks to make sure that you have the required clearance for a certain operation.
                    print("you have the required clearance for this")
                    #All inputs to eval have been checked. 
                    #Operation was validated earlier. n1 and n2 were confirmed as ints earlier.
                    #This means that we can cast them to a string and have eval do the math for us with the operation.
                    #If eval would fail because of inputs, the method would not reach this point in the code.
                    return eval(str(n1) + operation + str(n2))
                else:
                    print("You do not have the required clearance for this.")
        else: 
            print("The requested operation does not exist. Please choose '+', '-', or '*' ")
    else: 
        print("Please choose integer numbers to operate on.")
        
    #Returns this negative float upon failure.
    #Known error value because it cannot be reached with addition subtraction and multiplication of integers.
    return -10e-8  
    

In [495]:
addUser("userA","password",2)

admin, would you like to set userA's clearance level? They requested a clearance of 2.


True

In [496]:
changeClearanceLevel("admin","adminPassword","userA",2)

True

In [502]:
intArithmetic("userA","password","+",3,5)

you have the required clearance for this


8

In [498]:
login("admin","adminPassword")

5

In [499]:
database

{'admin': [b'$2b$12$qQXlHnrsKcs//cGUnSi86.y5mr7seVth2LcR5Al1xUvKhrsndFY3u',
  b'$2b$12$qQXlHnrsKcs//cGUnSi86.',
  5],
 'userA': [b'$2b$12$bn3QkymxmPgpp44sTVyOfOUdCs5xnOdNvD4PhqHKQGphtE1qLOFDm',
  b'$2b$12$bn3QkymxmPgpp44sTVyOfO',
  2]}